In [1]:
import torch
import numpy as np

from utils import Dataset, setup_seed
from models import NeuMF, ConvNCF
from train import train

In [2]:
dataset_path = 'data/lastfm'

# Load the dataset
dataset = Dataset(dataset_path)
train_mat, test_ratings, test_negatives = dataset.trainMatrix, dataset.testRatings, dataset.testNegatives
print('Dataset: #user=%d, #item=%d, #train_pairs=%d, #test_pairs=%d' 
      % (dataset.num_users, dataset.num_items, train_mat.nnz, len(test_ratings)))

mat = train_mat.todense()
mat = 1 * (mat > 0)
if type(mat) is np.matrix: mat = np.asarray(mat)

Dataset: #user=518, #item=3488, #train_pairs=45654, #test_pairs=518


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

users_num, items_num = dataset.num_users, dataset.num_items

### NeuMF

In [3]:
lr = 0.01
weight_decay = 1e-6
embedding_size = 16
num_layers = 1
epochs = 32
batch_size = 1024
mode = 'hr'
topK = 10

hr_list, ndcg_list = [], []
for seed in range(4):
    setup_seed(seed)
    model = NeuMF(users_num, items_num, embedding_size, num_layers)
    model = model.to(device)
    hr, ndcg = train(model, lr, weight_decay, test_ratings, test_negatives, topK, mat, epochs, batch_size, mode, device)
    hr_list.append(hr)
    ndcg_list.append(ndcg)
    
print('lr: ', lr, 'embedding_size: ', embedding_size, 'num_layers: ', num_layers)
print('HR: %.4f' %np.mean(hr_list), 'NDCG: %.4f' %np.mean(ndcg_list))
print('=' * 100)

Init: HR = 0.1081, NDCG = 0.0481
epoch=0, loss=0.4089, HR=0.6236, NDCG=0.4315
epoch=1, loss=0.2865, HR=0.7104, NDCG=0.4912
epoch=2, loss=0.2903, HR=0.7317, NDCG=0.5109
epoch=3, loss=0.2774, HR=0.7452, NDCG=0.5183
epoch=4, loss=0.2432, HR=0.7394, NDCG=0.5303
epoch=5, loss=0.2278, HR=0.7413, NDCG=0.5149
End. Best HR = 0.7452, NDCG = 0.5183. 
----------------------------------------------------------------------------------------------------
Init: HR = 0.1062, NDCG = 0.0485
epoch=0, loss=0.4099, HR=0.5946, NDCG=0.4199
epoch=1, loss=0.3833, HR=0.6988, NDCG=0.4988
epoch=2, loss=0.2993, HR=0.7278, NDCG=0.5185
epoch=3, loss=0.2588, HR=0.7181, NDCG=0.5185
epoch=4, loss=0.2576, HR=0.7413, NDCG=0.5222
epoch=5, loss=0.2026, HR=0.7162, NDCG=0.5134
epoch=6, loss=0.2105, HR=0.7317, NDCG=0.5029
End. Best HR = 0.7413, NDCG = 0.5222. 
----------------------------------------------------------------------------------------------------
Init: HR = 0.0927, NDCG = 0.0389
epoch=0, loss=0.3876, HR=0.6255, NDC

### ConvNCF

In [4]:
lr = 0.001
weight_decay = 1e-6
embedding_size = 64
epochs = 32
batch_size = 1024
mode = 'hr'
topK = 10
Loss = 'BPR'

hr_list, ndcg_list = [], []
for seed in range(4):
    setup_seed(seed)
    model = ConvNCF(users_num=users_num, items_num=items_num, embedding_size=embedding_size, device=device)
    model = model.to(device)
    hr, ndcg = train(model, lr, weight_decay, test_ratings, test_negatives, topK, mat, epochs, batch_size, mode, device, Loss)
    hr_list.append(hr)
    ndcg_list.append(ndcg)
    
print('lr: ', lr, 'embedding_size: ', embedding_size)
print('HR: %.4f' %np.mean(hr_list), 'NDCG: %.4f' %np.mean(ndcg_list))
print('=' * 100)

Init: HR = 0.1023, NDCG = 0.0480
epoch=0, loss=414.5019, HR=0.2066, NDCG=0.1107
epoch=1, loss=399.1942, HR=0.4073, NDCG=0.2577
epoch=2, loss=299.6071, HR=0.5502, NDCG=0.3527
epoch=3, loss=260.5023, HR=0.5367, NDCG=0.3526
epoch=4, loss=256.6011, HR=0.5656, NDCG=0.3689
epoch=5, loss=249.1218, HR=0.6004, NDCG=0.3993
epoch=6, loss=212.5953, HR=0.6216, NDCG=0.4089
epoch=7, loss=192.2262, HR=0.6274, NDCG=0.4140
epoch=8, loss=187.5975, HR=0.6351, NDCG=0.4206
epoch=9, loss=206.3753, HR=0.6255, NDCG=0.4277
epoch=10, loss=186.7796, HR=0.6467, NDCG=0.4352
epoch=11, loss=170.9395, HR=0.6737, NDCG=0.4589
epoch=12, loss=129.2584, HR=0.6969, NDCG=0.4804
epoch=13, loss=145.8621, HR=0.6950, NDCG=0.4837
epoch=14, loss=117.9635, HR=0.7085, NDCG=0.4874
epoch=15, loss=103.7029, HR=0.7104, NDCG=0.4802
epoch=16, loss=118.5165, HR=0.7046, NDCG=0.4804
epoch=17, loss=77.0063, HR=0.7220, NDCG=0.4839
epoch=18, loss=85.6144, HR=0.7162, NDCG=0.4932
epoch=19, loss=51.0461, HR=0.7027, NDCG=0.4962
End. Best HR = 0.722